## Preprocessing

In [1]:
#import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
#  Import and read the charity_data.csv.
# import pandas as pd 
# application_df = pd.read_csv("Resources/charity_data.csv")
# application_df.head()

application_df = pd.read_csv('/gdrive/MyDrive/Colab/deep-learning-challenge/Resources/charity_data.csv')
application_df

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df["APPLICATION_TYPE"].value_counts()

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_counts[application_counts<500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df["CLASSIFICATION"].value_counts()

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df["CLASSIFICATION"].value_counts().loc[lambda x : x>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts<1000].index)


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1]

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
app_dummies = pd.get_dummies(application_df)
app_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [11]:
# Split our preprocessed data into our features and target arrays
X = app_dummies.drop('IS_SUCCESSFUL', axis=1).values
y = app_dummies['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Create a StandardScaler instances
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [32]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = X_train_scaled.shape[1]
input_features

43

In [37]:
import tensorflow as tf
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])

# X shape/ input feature/columns is 43 * 2/3 is 86-129 so chose 100 for the first hidden node
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 50
hidden_nodes_layer3 = 40

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 80)                3520      
                                                                 
 dense_24 (Dense)            (None, 50)                4050      
                                                                 
 dense_25 (Dense)            (None, 40)                2040      
                                                                 
 dense_26 (Dense)            (None, 1)                 41        
                                                                 
Total params: 9,651
Trainable params: 9,651
Non-trainable params: 0
_________________________________________________________________


In [38]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [39]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5688 - accuracy: 0.7241
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5545 - accuracy: 0.7313
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5521 - accuracy: 0.7318
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5499 - accuracy: 0.7324
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5492 - accuracy: 0.7345
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5483 - accuracy: 0.7341
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5472 - accuracy: 0.7344
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5471 - accuracy: 0.7358
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5459 - accuracy: 0.7360
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5459 - accura

In [40]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5612 - accuracy: 0.7270 - 549ms/epoch - 2ms/step
Loss: 0.5611888766288757, Accuracy: 0.7269970774650574


In [58]:
# Export our model to HDF5 file
nn.save('/gdrive/MyDrive/Colab/deep-learning-challenge/Resources/AlphabetSoupCharity.h5')


In [41]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 80
hidden_nodes_layer3 = 60
hidden_nodes_layer4 = 40

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# third hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 100)               4400      
                                                                 
 dense_28 (Dense)            (None, 80)                8080      
                                                                 
 dense_29 (Dense)            (None, 60)                4860      
                                                                 
 dense_30 (Dense)            (None, 40)                2440      
                                                                 
 dense_31 (Dense)            (None, 1)                 41        
                                                                 
Total params: 19,821
Trainable params: 19,821
Non-trainable params: 0
_________________________________________________________________


In [42]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [43]:
# Train the model
fit_model = nn2.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5677 - accuracy: 0.7217
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5555 - accuracy: 0.7306
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5523 - accuracy: 0.7324
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5504 - accuracy: 0.7327
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5496 - accuracy: 0.7341
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5488 - accuracy: 0.7341
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5478 - accuracy: 0.7344
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5469 - accuracy: 0.7356
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5468 - accuracy: 0.7356
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5461 - accura

In [44]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5786 - accuracy: 0.7249 - 460ms/epoch - 2ms/step
Loss: 0.5785872340202332, Accuracy: 0.7248979806900024


In [59]:
# Export our model to HDF5 file
nn2.save('/gdrive/MyDrive/Colab/deep-learning-challenge/Resources/AlphabetSoupCharity_Optimization.h5')


In [50]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 80
hidden_nodes_layer3 = 60
hidden_nodes_layer4 = 40

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_42 (Dense)            (None, 100)               4400      
                                                                 
 dense_43 (Dense)            (None, 80)                8080      
                                                                 
 dense_44 (Dense)            (None, 60)                4860      
                                                                 
 dense_45 (Dense)            (None, 40)                2440      
                                                                 
 dense_46 (Dense)            (None, 1)                 41        
                                                                 
Total params: 19,821
Trainable params: 19,821
Non-trainable params: 0
_________________________________________________________________


In [51]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [52]:
# Train the model
fit_model = nn3.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5676 - accuracy: 0.7234
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5549 - accuracy: 0.7291
Epoch 3/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5531 - accuracy: 0.7325
Epoch 4/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5507 - accuracy: 0.7334
Epoch 5/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5495 - accuracy: 0.7344
Epoch 6/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5487 - accuracy: 0.7343
Epoch 7/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5474 - accuracy: 0.7346
Epoch 8/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5470 - accuracy: 0.7360
Epoch 9/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5457 - accuracy: 0.7362
Epoch 10/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5457 - accura

In [53]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5782 - accuracy: 0.7245 - 451ms/epoch - 2ms/step
Loss: 0.5781909227371216, Accuracy: 0.7245481014251709


In [60]:
# Export our model to HDF5 file
nn3.save('/gdrive/MyDrive/Colab/deep-learning-challenge/Resources/AlphabetSoupCharity_Optimization2.h5')

In [54]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 120
hidden_nodes_layer2 = 90
hidden_nodes_layer3 = 80
hidden_nodes_layer4 = 60
hidden_nodes_layer5 = 50

nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh")
)


# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# third hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Fifth hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="relu"))

# Output layer
nn4.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn4.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_47 (Dense)            (None, 120)               5280      
                                                                 
 dense_48 (Dense)            (None, 90)                10890     
                                                                 
 dense_49 (Dense)            (None, 80)                7280      
                                                                 
 dense_50 (Dense)            (None, 60)                4860      
                                                                 
 dense_51 (Dense)            (None, 50)                3050      
                                                                 
 dense_52 (Dense)            (None, 1)                 51        
                                                                 
Total params: 31,411
Trainable params: 31,411
Non-tra

In [55]:
# Compile the model
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [56]:
# Train the model
fit_model = nn4.fit(X_train_scaled,y_train,epochs=250)

Epoch 1/250
804/804 [==============================] - 3s 2ms/step - loss: 0.5656 - accuracy: 0.7246
Epoch 2/250
804/804 [==============================] - 1s 2ms/step - loss: 0.5564 - accuracy: 0.7306
Epoch 3/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5535 - accuracy: 0.7320
Epoch 4/250
804/804 [==============================] - 1s 2ms/step - loss: 0.5517 - accuracy: 0.7332
Epoch 5/250
804/804 [==============================] - 1s 2ms/step - loss: 0.5513 - accuracy: 0.7335
Epoch 6/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5497 - accuracy: 0.7339
Epoch 7/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.7346
Epoch 8/250
804/804 [==============================] - 2s 3ms/step - loss: 0.5483 - accuracy: 0.7359
Epoch 9/250
804/804 [==============================] - 2s 2ms/step - loss: 0.5468 - accuracy: 0.7376
Epoch 10/250
804/804 [==============================] - 1s 2ms/step - loss: 0.5466 - accura

In [57]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5852 - accuracy: 0.7259 - 443ms/epoch - 2ms/step
Loss: 0.5851919054985046, Accuracy: 0.7259474992752075


In [63]:
# Export our model to HDF5 file
nn4.save('/gdrive/MyDrive/Colab/deep-learning-challenge/Resources/AlphabetSoupCharity_Optimization3.h5')